<a href="https://colab.research.google.com/github/kofiadom/Web-Scraping-with-Selenium-Colab-/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,304 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,650 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [2]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb

--2025-01-30 18:31:17--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.216.91, 173.194.216.136, 173.194.216.93, ...
Connecting to dl.google.com (dl.google.com)|173.194.216.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112524488 (107M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 107.31M   347MB/s    in 0.3s    

2025-01-30 18:31:17 (347 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [112524488/112524488]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded,

In [3]:
!pip install pandas

In [4]:
import pandas as pd
import time
from selenium import webdriver

##Create Driver

In [5]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [6]:
driver = driversetup()

In [18]:
# Go to the page that we want to scrape -- characters in the book page
page_url = "https://witcher.fandom.com/wiki/Category:Characters_in_the_stories"
# Reinitialize the driver if you get `NoSuchWindowException` error which indicates that the browser window has been closed
driver.get(page_url)

##Scraping

In [19]:
from selenium.webdriver.common.by import By
book_categories = driver.find_elements(By.CLASS_NAME, 'category-page__member-link')

In [20]:
book_categories

[<selenium.webdriver.remote.webelement.WebElement (session="a11d9b028f4467fc84ac008e31f2f3f9", element="f.BA37F33443B904FD1590C3B82B30956C.d.4096849953722FE5C6D02D53CCEDD83D.e.380")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a11d9b028f4467fc84ac008e31f2f3f9", element="f.BA37F33443B904FD1590C3B82B30956C.d.4096849953722FE5C6D02D53CCEDD83D.e.381")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a11d9b028f4467fc84ac008e31f2f3f9", element="f.BA37F33443B904FD1590C3B82B30956C.d.4096849953722FE5C6D02D53CCEDD83D.e.382")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a11d9b028f4467fc84ac008e31f2f3f9", element="f.BA37F33443B904FD1590C3B82B30956C.d.4096849953722FE5C6D02D53CCEDD83D.e.383")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a11d9b028f4467fc84ac008e31f2f3f9", element="f.BA37F33443B904FD1590C3B82B30956C.d.4096849953722FE5C6D02D53CCEDD83D.e.384")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a11d9b028f4467fc84ac

In [10]:
#book_categories[0].text

'Category:Baptism of Fire characters'

In [11]:
#book_categories[0].get_attribute('href')

'https://witcher.fandom.com/wiki/Category:Baptism_of_Fire_characters'

In [12]:
# Go into the url of the first book item
#driver.get(book_categories[0].get_attribute('href'))

In [13]:
#character_elements = driver.find_elements(By.CLASS_NAME, 'category-page__member-link')

In [14]:
#character_elements[0].text

'Adalia'

In [21]:
from selenium.common.exceptions import StaleElementReferenceException

books = []
for i in range(len(book_categories)):
    try:
        # Re-fetch the element to avoid StaleElementReferenceException
        book_categories = driver.find_elements(By.CLASS_NAME, 'category-page__member-link')[i]
        book_url = book_categories.get_attribute('href')
        book_name = book_categories.text
        books.append({'book_name': book_name, 'book_url': book_url})
    except StaleElementReferenceException:
        # Handle the exception if needed
        print(f"StaleElementReferenceException encountered for element {i}")

In [22]:
books

[{'book_name': 'Category:Baptism of Fire characters',
  'book_url': 'https://witcher.fandom.com/wiki/Category:Baptism_of_Fire_characters'},
 {'book_name': 'Category:Blood of Elves characters',
  'book_url': 'https://witcher.fandom.com/wiki/Category:Blood_of_Elves_characters'},
 {'book_name': "Godamba Thaess'en",
  'book_url': 'https://witcher.fandom.com/wiki/Godamba_Thaess%27en'},
 {'book_name': 'Category:Season of Storms characters',
  'book_url': 'https://witcher.fandom.com/wiki/Category:Season_of_Storms_characters'},
 {'book_name': 'Category:Something Ends, Something Begins characters',
  'book_url': 'https://witcher.fandom.com/wiki/Category:Something_Ends,_Something_Begins_characters'},
 {'book_name': 'Category:Sword of Destiny characters',
  'book_url': 'https://witcher.fandom.com/wiki/Category:Sword_of_Destiny_characters'},
 {'book_name': 'Category:Szpony i kły characters',
  'book_url': 'https://witcher.fandom.com/wiki/Category:Szpony_i_k%C5%82y_characters'},
 {'book_name': 'Cat

In [23]:
character_list = []

for book in books:
    driver.get(book['book_url'])
    character_elements = driver.find_elements(By.CLASS_NAME, 'category-page__member-link')
    for character_element in character_elements:
        character_list.append({'book_name': book['book_name'], 'character_name': character_element.text})

In [24]:
pd.DataFrame(character_list)

,book_name,character_name
0,Category:Baptism of Fire characters,Adalia
1,Category:Baptism of Fire characters,Adela
2,Category:Baptism of Fire characters,Aen Saevherne
3,Category:Baptism of Fire characters,Aevenien
4,Category:Baptism of Fire characters,Aglaïs
...,...,...
1271,Category:Time of Contempt characters,Yanna of Murivel
1272,Category:Time of Contempt characters,Yarpen Zigrin
1273,Category:Time of Contempt characters,Yennefer of Vengerberg
1274,Category:Time of Contempt characters,Yiolenta Suarez
